In [5]:
import pandas as pd

In [6]:
df=pd.read_csv(r'\Users\nice\Desktop\sales-forecasting-project (1)\sales-forecasting-project\data\processed\processed_data.csv')
df

,Date,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Demand Forecast,Price,Discount,Weather Condition,Holiday/Promotion,Competitor Pricing,Seasonality,Sales,year,month,dayofweek
0,2022-01-01,S001,P0001,Groceries,North,231,127,55,135.47,33.50,20,Rainy,0,29.69,Winter,4234.50,2022,1,5
1,2022-01-01,S001,P0002,Toys,South,204,150,66,144.04,63.01,20,Sunny,0,66.16,Winter,9431.50,2022,1,5
2,2022-01-01,S001,P0003,Toys,West,102,65,51,74.02,27.99,10,Sunny,1,31.32,Winter,1809.35,2022,1,5
3,2022-01-01,S001,P0004,Toys,North,469,61,164,62.18,32.72,10,Cloudy,1,34.74,Winter,1985.92,2022,1,5
4,2022-01-01,S001,P0005,Electronics,East,166,14,135,9.26,73.64,0,Sunny,0,68.95,Winter,1030.96,2022,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67995,2024-01-01,S005,P0016,Furniture,East,96,8,127,18.46,73.73,20,Snowy,0,72.45,Winter,569.84,2024,1,0
67996,2024-01-01,S005,P0017,Toys,North,313,51,101,48.43,82.57,10,Cloudy,0,83.78,Winter,4201.07,2024,1,0
67997,2024-01-01,S005,P0018,Clothing,West,278,36,151,39.65,11.11,10,Rainy,0,10.91,Winter,389.96,2024,1,0
67998,2024-01-01,S005,P0019,Toys,East,374,264,21,270.52,53.14,20,Rainy,0,55.80,Winter,14008.96,2024,1,0


In [7]:
df.loc[df["month"]==1,"Seasonality"].unique()

array(['Winter'], dtype=object)

In [1]:
import dash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
import pandas as pd
import plotly.express as px
import os
from datetime import datetime as dt

# Initialize the app with external stylesheets
external_stylesheets = [
    'https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.1/css/all.min.css',
    'https://fonts.googleapis.com/css2?family=Roboto:wght@300;400;500;700&display=swap'
]

app = dash.Dash(__name__, 
                suppress_callback_exceptions=True,
                external_stylesheets=external_stylesheets,
                meta_tags=[{'name': 'viewport', 
                          'content': 'width=device-width, initial-scale=1.0'}])
server = app.server

# Custom CSS styles
custom_css = {
    'navbar': {
        'padding': '15px 30px',
        'backgroundColor': '#2c3e50',
        'boxShadow': '0 2px 5px rgba(0,0,0,0.1)',
        'marginBottom': '30px'
    },
    'navLink': {
        'color': 'white',
        'marginRight': '25px',
        'textDecoration': 'none',
        'fontWeight': '500',
        'fontSize': '16px',
        'transition': 'all 0.3s ease'
    },
    'navLinkHover': {
        'color': '#f39c12',
        'textDecoration': 'none'
    },
    'card': {
        'borderRadius': '10px',
        'boxShadow': '0 4px 8px rgba(0,0,0,0.1)',
        'padding': '25px',
        'backgroundColor': 'white',
        'marginBottom': '30px'
    },
    'metricCard': {
        'borderRadius': '8px',
        'boxShadow': '0 2px 4px rgba(0,0,0,0.05)',
        'padding': '15px',
        'backgroundColor': 'white',
        'textAlign': 'center',
        'borderTop': '4px solid #3498db'
    },
    'title': {
        'color': '#2c3e50',
        'marginBottom': '25px',
        'fontWeight': '700'
    },
    'filterRow': {
        'backgroundColor': '#f8f9fa',
        'padding': '15px',
        'borderRadius': '8px',
        'marginBottom': '25px'
    }
}

# Load your data
def load_data():
    path = r"\Users\nice\Desktop\sales-forecasting-project (1)\sales-forecasting-project\data\processed\processed_data.csv"
    df = pd.read_csv(path)
    
    # Data preprocessing
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df['month'] = df['Date'].dt.month
    df['year'] = df['Date'].dt.year
    df['dayofweek'] = df['Date'].dt.dayofweek
    
    def get_season(month):
        if month in [12, 1, 2]: return 'Winter'
        elif month in [3, 4, 5]: return 'Spring'
        elif month in [6, 7, 8]: return 'Summer'
        else: return 'Autumn'
    df['Seasonality'] = df['month'].apply(get_season)
    
    df = df[df['Demand Forecast'] >= 0]
    return df

df = load_data()

# Define the app layout with navigation
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    
    # Navigation bar with improved styling
    html.Nav([
        html.Div([
            html.Span("Retail Analytics", style={
                'color': 'white',
                'fontWeight': 'bold',
                'fontSize': '20px',
                'marginRight': '40px'
            }),
            dcc.Link(html.Div([
                html.I(className="fas fa-home"), 
                html.Span(" Description", style={'marginLeft': '8px'})
            ], style={'display': 'flex', 'alignItems': 'center'}), 
            href='/', style=custom_css['navLink']),
            
            dcc.Link(html.Div([
                html.I(className="fas fa-database"), 
                html.Span(" Dataset", style={'marginLeft': '8px'})
            ], style={'display': 'flex', 'alignItems': 'center'}), 
            href='/dataset', style=custom_css['navLink']),
            
            dcc.Link(html.Div([
                html.I(className="fas fa-chart-line"), 
                html.Span(" Analysis", style={'marginLeft': '8px'})
            ], style={'display': 'flex', 'alignItems': 'center'}), 
            href='/analysis', style=custom_css['navLink'])
        ], style={'display': 'flex', 'alignItems': 'center'})
    ], style=custom_css['navbar']),
    
    # Page content
    html.Div(id='page-content', style={'padding': '0 30px'})
])

# Description Page Layout with improved design
description_layout = html.Div([
    html.Div([
        html.H1('Retail Demand Forecasting Project', 
               style={**custom_css['title'], 'textAlign': 'center'}),
        
        html.Div([
            html.Div([
                html.H2('Project Description', style={'color': '#3498db'}),
                html.P("""
                    This interactive dashboard provides comprehensive insights into retail store inventory 
                    and demand forecasting. The project helps retailers optimize inventory levels based on 
                    sales patterns, demand forecasts, and external factors like seasonality and promotions.
                """, style={'lineHeight': '1.6'}),
                
                html.H3('Key Features:', style={'marginTop': '25px', 'color': '#3498db'}),
                html.Ul([
                    html.Li('Interactive visualization of sales trends', style={'marginBottom': '8px'}),
                    html.Li('Inventory vs demand analysis with forecasting', style={'marginBottom': '8px'}),
                    html.Li('Seasonal pattern identification', style={'marginBottom': '8px'}),
                    html.Li('Regional performance comparison', style={'marginBottom': '8px'}),
                    html.Li('Promotion effectiveness analysis', style={'marginBottom': '8px'})
                ], style={'paddingLeft': '20px', 'marginBottom': '25px'}),
                
                html.H3('Data Sources:', style={'color': '#3498db'}),
                html.P("""
                    The dataset contains detailed retail store inventory information including:
                """, style={'marginBottom': '10px'}),
                html.Ul([
                    html.Li('Store and product details', style={'marginBottom': '8px'}),
                    html.Li('Inventory levels and units sold', style={'marginBottom': '8px'}),
                    html.Li('Demand forecasts and historical data', style={'marginBottom': '8px'}),
                    html.Li('Pricing, discount, and promotion information', style={'marginBottom': '8px'}),
                    html.Li('Weather conditions and seasonal factors', style={'marginBottom': '8px'})
                ], style={'paddingLeft': '20px'}),
                
                html.H3('How to Use:', style={'marginTop': '25px', 'color': '#3498db'}),
                html.P("""
                    Navigate between pages using the menu at the top to access different features:
                """, style={'marginBottom': '10px'}),
                html.Ul([
                    html.Li(html.Strong('Description:'), ' Project overview and documentation', 
                           style={'marginBottom': '8px'}),
                    html.Li(html.Strong('Dataset:'), ' View and explore the raw data', 
                           style={'marginBottom': '8px'}),
                    html.Li(html.Strong('Analysis:'), ' Interactive visualizations and dashboards', 
                           style={'marginBottom': '8px'})
                ], style={'paddingLeft': '20px', 'marginBottom': '25px'}),
                
                html.H3('Project Team:', style={'color': '#3498db'}),
                html.P("""
                    This project was developed as part of a data science course by:
                """, style={'marginBottom': '10px'}),
                html.Div([
                    html.Span("Mohamed Salah", style={'marginRight': '15px'}),
                    html.Span("Abdelwahab Amr", style={'marginRight': '15px'}),
                    html.Span("Khaled Hany", style={'marginRight': '15px'}),
                    html.Span("Omar Ahmed", style={'marginRight': '15px'}),
                    html.Span("Hatem", style={'marginRight': '15px'}),
                    html.Span("Bakhom", style={'marginRight': '15px'})
                ], style={'fontStyle': 'italic'})
            ], style={'padding': '25px'})
        ], style=custom_css['card'])
    ], style={'maxWidth': '1000px', 'margin': '0 auto'})
])

# Dataset Page Layout with improved design
dataset_layout = html.Div([
    html.Div([
        html.H1('Retail Store Inventory Dataset', 
               style={**custom_css['title'], 'textAlign': 'center'}),
        
        html.Div([
            html.Div([
                html.Div([
                    html.Label('Select number of rows to display:', 
                             style={'marginBottom': '8px', 'fontWeight': '500'}),
                    dcc.Dropdown(
                        id='row-selector',
                        options=[
                            {'label': '100 rows', 'value': 100},
                            {'label': '500 rows', 'value': 500},
                            {'label': '1000 rows', 'value': 1000},
                            {'label': 'All rows', 'value': -1}
                        ],
                        value=100,
                        clearable=False,
                        style={'width': '100%'}
                    )
                ], style={'width': '200px', 'marginBottom': '25px'}),
                
                html.Div([
                    html.Label('Quick Statistics:', 
                             style={'marginBottom': '8px', 'fontWeight': '500'}),
                    dash_table.DataTable(
                        id='stats-table',
                        style_cell={
                            'textAlign': 'left',
                            'padding': '10px',
                            'fontFamily': 'Roboto'
                        },
                        style_header={
                            'backgroundColor': '#3498db',
                            'color': 'white',
                            'fontWeight': 'bold'
                        },
                        style_data_conditional=[
                            {
                                'if': {'row_index': 'odd'},
                                'backgroundColor': 'rgb(248, 248, 248)'
                            }
                        ]
                    )
                ], style={'marginBottom': '30px'}),
                
                html.Div([
                    dash_table.DataTable(
                        id='data-table',
                        columns=[{'name': col, 'id': col} for col in df.columns],
                        page_size=20,
                        style_table={
                            'overflowX': 'auto',
                            'borderRadius': '8px',
                            'boxShadow': '0 2px 4px rgba(0,0,0,0.05)'
                        },
                        style_cell={
                            'height': 'auto',
                            'minWidth': '100px',
                            'maxWidth': '200px',
                            'whiteSpace': 'normal',
                            'fontFamily': 'Roboto',
                            'padding': '10px',
                            'border': '1px solid #eee'
                        },
                        style_header={
                            'backgroundColor': '#2c3e50',
                            'color': 'white',
                            'fontWeight': 'bold'
                        },
                        filter_action='native',
                        sort_action='native',
                        sort_mode='multi',
                        page_action='native'
                    )
                ])
            ], style={'padding': '25px'})
        ], style=custom_css['card'])
    ], style={'maxWidth': '1200px', 'margin': '0 auto'})
])

# Analysis Page Layout with improved design
analysis_layout = html.Div([
    html.Div([
        html.H1("Retail Demand Forecasting Dashboard", 
               style={**custom_css['title'], 'textAlign': 'center'}),
        
        # Filters row with improved design
        html.Div([
            html.Div([
                html.Label('Date Range:', style={'marginBottom': '8px', 'fontWeight': '500'}),
                dcc.DatePickerRange(
                    id='date-range',
                    min_date_allowed=df['Date'].min(),
                    max_date_allowed=df['Date'].max(),
                    start_date=df['Date'].min(),
                    end_date=df['Date'].max(),
                    display_format='YYYY-MM-DD',
                    style={'width': '100%'}
                )
            ], style={'width': '30%', 'display': 'inline-block'}),
            
            html.Div([
                html.Label('Categories:', style={'marginBottom': '8px', 'fontWeight': '500'}),
                dcc.Dropdown(
                    id='category-selector',
                    options=[{'label': cat, 'value': cat} for cat in df['Category'].unique()],
                    value=df['Category'].unique(),
                    multi=True,
                    placeholder="Select Categories",
                    style={'width': '100%'}
                )
            ], style={'width': '30%', 'display': 'inline-block', 'marginLeft': '5%'}),
            
            html.Div([
                html.Label('Regions:', style={'marginBottom': '8px', 'fontWeight': '500'}),
                dcc.Dropdown(
                    id='region-selector',
                    options=[{'label': reg, 'value': reg} for reg in df['Region'].unique()],
                    value=df['Region'].unique(),
                    multi=True,
                    placeholder="Select Regions",
                    style={'width': '100%'}
                )
            ], style={'width': '30%', 'display': 'inline-block', 'marginLeft': '5%'})
        ], style=custom_css['filterRow']),

        # X/Y-axis selectors for scatter plot
        html.Div([
            html.Div([
                html.Label('Scatter X-axis:', style={'marginBottom': '8px', 'fontWeight': '500'}),
                dcc.Dropdown(
                    id='x-axis-selector',
                    options=[{'label': col, 'value': col} for col in df.columns],
                    value='Inventory Level',
                    clearable=False,
                    style={'width': '100%'}
                )
            ], style={'width': '30%', 'display': 'inline-block'}),
            
            html.Div([
                html.Label('Scatter Y-axis:', style={'marginBottom': '8px', 'fontWeight': '500'}),
                dcc.Dropdown(
                    id='y-axis-selector',
                    options=[{'label': col, 'value': col} for col in df.columns],
                    value='Units Sold',
                    clearable=False,
                    style={'width': '100%'}
                )
            ], style={'width': '30%', 'display': 'inline-block', 'marginLeft': '5%'})
        ], style=custom_css['filterRow']),

        # Bar chart axis selectors (ADDED INSIDE THE MAIN CONTAINER)
        html.Div([
            html.Div([
                html.Label('Bar Chart X-axis:', style={'marginBottom': '8px', 'fontWeight': '500'}),
                dcc.Dropdown(
                    id='bar-x-axis-selector',
                    options=[{'label': col, 'value': col} for col in df.columns],
                    value='Region',
                    clearable=False,
                    style={'width': '100%'}
                )
            ], style={'width': '30%', 'display': 'inline-block'}),

            html.Div([
                html.Label('Bar Chart Y-axis:', style={'marginBottom': '8px', 'fontWeight': '500'}),
                dcc.Dropdown(
                    id='bar-y-axis-selector',
                    options=[{'label': col, 'value': col} for col in df.columns],
                    value='Units Sold',
                    clearable=False,
                    style={'width': '100%'}
                )
            ], style={'width': '30%', 'display': 'inline-block', 'marginLeft': '5%'})
            
        ], style=custom_css['filterRow']),
        # Metrics cards with improved design
        # Add this section with your other axis selectors
html.Div([
    html.Div([
        html.Label('Line Chart Y-axis:', style={'marginBottom': '8px', 'fontWeight': '500'}),
        dcc.Dropdown(
            id='line-y-axis-selector',
            options=[{'label': col, 'value': col} for col in df.columns],
            value='Units Sold',
            clearable=False,
            style={'width': '100%'}
        )
    ], style={'width': '30%', 'display': 'inline-block'}),
    
    html.Div([
        html.Label('Box Plot X-axis:', style={'marginBottom': '8px', 'fontWeight': '500'}),
        dcc.Dropdown(
            id='box-x-axis-selector',
            options=[{'label': col, 'value': col} for col in df.columns],
            value='Category',
            clearable=False,
            style={'width': '100%'}
        )
    ], style={'width': '30%', 'display': 'inline-block', 'marginLeft': '5%'}),
    
    # Add an empty div to maintain layout if needed
    html.Div([], style={'width': '30%', 'display': 'inline-block', 'marginLeft': '5%'})
], style=custom_css['filterRow']),
        html.Div([
            html.Div([
                html.Div([
                    html.Div([
                        html.I(className="fas fa-shopping-cart", 
                              style={'fontSize': '24px', 'color': '#3498db'}),
                        html.H3("Total Sales", style={'margin': '10px 0 5px 0'}),
                        html.H4(id='total-sales', style={'margin': '0', 'color': '#2c3e50'})
                    ])
                ], style={**custom_css['metricCard'], 'borderTop': '4px solid #3498db'})
            ], style={'width': '23%', 'display': 'inline-block', 'marginRight': '2%'}),
            
            html.Div([
                html.Div([
                    html.Div([
                        html.I(className="fas fa-boxes", 
                              style={'fontSize': '24px', 'color': '#e74c3c'}),
                        html.H3("Avg. Inventory", style={'margin': '10px 0 5px 0'}),
                        html.H4(id='avg-inventory', style={'margin': '0', 'color': '#2c3e50'})
                    ])
                ], style={**custom_css['metricCard'], 'borderTop': '4px solid #e74c3c'})
            ], style={'width': '23%', 'display': 'inline-block', 'marginRight': '2%'}),
            
            html.Div([
                html.Div([
                    html.Div([
                        html.I(className="fas fa-bullseye", 
                              style={'fontSize': '24px', 'color': '#2ecc71'}),
                        html.H3("Forecast Accuracy", style={'margin': '10px 0 5px 0'}),
                        html.H4(id='forecast-accuracy', style={'margin': '0', 'color': '#2c3e50'})
                    ])
                ], style={**custom_css['metricCard'], 'borderTop': '4px solid #2ecc71'})
            ], style={'width': '23%', 'display': 'inline-block', 'marginRight': '2%'}),
            
            html.Div([
                html.Div([
                    html.Div([
                        html.I(className="fas fa-percentage", 
                              style={'fontSize': '24px', 'color': '#f39c12'}),
                        html.H3("Avg. Discount", style={'margin': '10px 0 5px 0'}),
                        html.H4(id='avg-discount', style={'margin': '0', 'color': '#2c3e50'})
                    ])
                ], style={**custom_css['metricCard'], 'borderTop': '4px solid #f39c12'})
            ], style={'width': '23%', 'display': 'inline-block'})
        ], style={'marginBottom': '30px'}),
        
        
        # Charts with improved layout
        html.Div([
            
            html.Div([
                dcc.Loading(
                    id="loading-scatter",
                    type="circle",
                    children=[dcc.Graph(id='scatter-plot')]
                )
            ], style={**custom_css['card'], 'width': '48%', 'display': 'inline-block', 'marginRight': '4%'}),
            
            html.Div([
                dcc.Loading(
                    id="loading-bar",
                    type="circle",
                    children=[dcc.Graph(id='bar-chart')]
                )
            ], style={**custom_css['card'], 'width': '48%', 'display': 'inline-block'})
        ], style={'marginBottom': '30px'}),
        
        html.Div([
            html.Div([
                dcc.Loading(
                    id="loading-line",
                    type="circle",
                    children=[dcc.Graph(id='line-chart')]
                )
            ], style={**custom_css['card'], 'width': '48%', 'display': 'inline-block', 'marginRight': '4%'}),
            
            html.Div([
                dcc.Loading(
                    id="loading-box",
                    type="circle",
                    children=[dcc.Graph(id='box-plot')]
                )
            ], style={**custom_css['card'], 'width': '48%', 'display': 'inline-block'})
        ])
        # Add this block before the final closing of the main container Div
    ], style={'maxWidth': '1200px', 'margin': '0 auto'})
])

# Callback to switch between pages
@app.callback(
    Output('page-content', 'children'),
    [Input('url', 'pathname')]
)
def display_page(pathname):
    if pathname == '/dataset':
        return dataset_layout
    elif pathname == '/analysis':
        return analysis_layout
    else:
        return description_layout  

# Dataset page callbacks
@app.callback(
    [Output('data-table', 'data'),
     Output('stats-table', 'data'),
     Output('stats-table', 'columns')],
    [Input('row-selector', 'value')]
)
def update_table(rows):
    if rows == -1:
        display_df = df
    else:
        display_df = df.head(rows)
    
    # Create summary statistics
    stats_df = df.describe().reset_index()
    stats_columns = [{'name': 'Statistic', 'id': 'index'}] + [{'name': col, 'id': col} for col in stats_df.columns[1:]]
    
    return display_df.to_dict('records'), stats_df.to_dict('records'), stats_columns

@app.callback(
   [Output('scatter-plot', 'figure'),
    Output('bar-chart', 'figure'),
    Output('line-chart', 'figure'),
    Output('box-plot', 'figure'),
    Output('total-sales', 'children'),
    Output('avg-inventory', 'children'),
    Output('forecast-accuracy', 'children'),
    Output('avg-discount', 'children')],
    [Input('date-range', 'start_date'),
     Input('date-range', 'end_date'),
     Input('category-selector', 'value'),
     Input('region-selector', 'value'),
     Input('x-axis-selector', 'value'),
     Input('y-axis-selector', 'value'),
     Input('bar-x-axis-selector', 'value'),
     Input('bar-y-axis-selector', 'value'),
     Input('line-y-axis-selector', 'value'),
     Input('box-x-axis-selector', 'value')]
)

def update_analysis(start_date, end_date, selected_categories, selected_regions, 
                    scatter_x_axis, scatter_y_axis, bar_x_axis, bar_y_axis, line_y_axis,box_x_axis):

    filtered_df = df[
        (df['Date'] >= start_date) &
        (df['Date'] <= end_date) &
        (df['Category'].isin(selected_categories)) &
        (df['Region'].isin(selected_regions))
    ]
    
    if filtered_df.empty:
        empty_fig = px.scatter(title="No data available for selected filters")
        empty_fig.update_layout(
            plot_bgcolor='white',
            paper_bgcolor='white',
            xaxis={'visible': False},
            yaxis={'visible': False},
            annotations=[{
                'text': 'No data available',
                'showarrow': False,
                'font': {'size': 16}
            }]
        )
        return [empty_fig] * 4 + ["N/A"] * 4
    
    # Calculate metrics
    total_sales = filtered_df['Units Sold'].sum()
    avg_inventory = filtered_df['Inventory Level'].mean()
    
    filtered_df = filtered_df[filtered_df['Units Sold'] > 0]
    if len(filtered_df) > 0:
        mape = (abs(filtered_df['Units Sold'] - filtered_df['Demand Forecast']) / 
                filtered_df['Units Sold']).mean() * 100
        forecast_accuracy = f"{100 - mape:.1f}%"
    else:
        forecast_accuracy = "N/A"
    
    avg_discount = filtered_df['Discount'].mean()
    
    # Create figures with consistent styling
    scatter_fig = px.scatter(
        filtered_df, 
        x = scatter_x_axis,
        y = scatter_y_axis,
        color='Category', 
        title='Inventory vs Sales',
        template='plotly_white'
    )
    scatter_fig.update_layout(
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        font={'family': 'Roboto'}
    )
    
    bar_fig = px.bar(
        filtered_df, 
        x= bar_x_axis,
        y= bar_y_axis, 
        color='Category' if bar_x_axis != 'Category' else None, 
        barmode='group', 
        title=f'{bar_y_axis} by {bar_x_axis}',
        template='plotly_white'
    )
    bar_fig.update_layout(
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        font={'family': 'Roboto'}
    )
    
    line_fig = px.line(
        filtered_df.groupby('Date')[line_y_axis].sum().reset_index(), 
        x='Date', 
        y=line_y_axis, 
        title=f'{line_y_axis} Over Time',
        template='plotly_white'
    )
    line_fig.update_layout(
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        font={'family': 'Roboto'}
    )
    
    box_fig = px.box(
        filtered_df, 
        x=box_x_axis, 
        y='Units Sold', 
        title=f'Sales Distribution by {box_x_axis}',
        template='plotly_white'
    )
    box_fig.update_layout(
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        font={'family': 'Roboto'}
    )

    return (
        scatter_fig, 
        bar_fig, 
        line_fig, 
        box_fig, 
        f"{total_sales:,}", 
        f"{avg_inventory:,.0f}", 
        forecast_accuracy, 
        f"{avg_discount:.1f}%"
    )

if __name__ == '__main__':
    app.run(debug=True, port=8040)